# Process Input scATAC-seq


In [ ]:
import random
import sys
import numpy as np
import pysam
import pandas as pd
import os
import itertools
import scipy
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle


In [ ]:
ct = 'imr90'

In [ ]:
frag_path = './rawdata/scatac/fragments.tsv.gz'
valid_barcode = pd.read_csv('./rawdata/scatac/archr_filtered_barcode.csv', index_col = 0)
chrom_size = pd.read_csv('../genome/hg38_chrNameLength.txt', 
                         sep = '\t', header = None, index_col = 0)

valid_chrom = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5','chr6','chr7',
               'chr8', 'chr9','chr10','chr11','chr12','chr13','chr14',
               'chr15','chr16','chr17','chr18','chr19','chr20',
               'chr21','chr22','chrX']


## Process scATAC at 500bp (for co-accessibility input)

In [ ]:
frag = pd.read_csv(frag_path, sep = '\t', header=None)
dist = frag[2]-frag[1]

# Option to filter cell barcodes using cluster assignments or number of fragments
# valid_barcode = valid_barcode[valid_barcode['nFrags'] > 1000]
valid_barcode = valid_barcode.index
# Filter for valid barcodes only
frag = frag[frag[3].isin(valid_barcode)]


In [ ]:
frag_stacked = pd.concat([frag[[0,1,3,4]],frag[[0,2,3,4]].rename(columns={2:1})], ignore_index=True)
frag_stacked[4] = 1 #binarize count
frag_stacked[1] = frag_stacked[1]//500 # aggregate counts per 500bp tile

barcodes, barcode_id = np.unique(frag_stacked[3], return_inverse=True)
frag_stacked[5] = barcode_id


In [ ]:
tile_dict = {}
for chrom in list(set(frag_stacked[0])):
    if chrom in valid_chrom:
        frag_grouped = frag_stacked[frag_stacked[0]==chrom][[1,5,4]].groupby([5,1]).sum().reset_index()
        frag_grouped = csr_matrix((frag_grouped[4].astype(np.float64), (frag_grouped[5].values, frag_grouped[1].values)), shape=(barcodes.shape[0], chrom_size.loc[chrom].values[0]//500+1))
        tile_dict[chrom] = frag_grouped
        print(chrom)
        

In [ ]:
pickle.dump(tile_dict, open("./data/scatac/processed_input/{}_tile_500bp_dict.p".format(ct), "wb"))
np.save("./data/scatac/processed_input/{}_tile_500bp_barcode.npy".format(ct),barcodes.astype(str))


## Pseudo-bulk (for evaluation)

In [ ]:
frag_stacked = pd.concat([frag[[0,1,4]],frag[[0,2,4]].rename(columns={2:1})], ignore_index=True)
frag_stacked[4] = 1 #binarize counte
frag_stacked[1] = frag_stacked[1]//50 # aggregate counts per 50bp tile


In [ ]:
tile_dict = {}
for chrom in list(set(frag_stacked[0])):
    if chrom in valid_chrom:
        frag_grouped = frag_stacked[frag_stacked[0]==chrom][[1,4]].groupby([1]).sum().reset_index()
        frag_grouped = pd.DataFrame(frag_grouped[4].values, index = frag_grouped[1].values).reindex(np.arange(0,chrom_size.loc[chrom].values[0]//50+1)).fillna(0).values
        tile_dict[chrom] = frag_grouped
        

In [ ]:
import pickle
pickle.dump(tile_dict, open("./data/scatac/processed_input/{}_tile_pbulk_50bp_dict.p".format(ct), "wb"))


## Process scATAC at 50bp (for training)

In [ ]:
frag_stacked = pd.concat([frag[[0,1,3,4]],frag[[0,2,3,4]].rename(columns={2:1})], ignore_index=True)
frag_stacked[4] = 1 #binarize counte
frag_stacked[1] = frag_stacked[1]//50 # aggregate counts per 50bp tile

barcodes, barcode_id = np.unique(frag_stacked[3], return_inverse=True)
frag_stacked[5] = barcode_id


In [ ]:
tile_dict = {}
for chrom in valid_chrom:
    frag_grouped = frag_stacked[frag_stacked[0]==chrom][[1,5,4]].groupby([5,1]).sum().reset_index()
    frag_grouped = csr_matrix((frag_grouped[4].astype(np.float64), (frag_grouped[5], frag_grouped[1])), shape=(barcodes.shape[0], chrom_size.loc[chrom].values[0]//tile_size+1))
    tile_dict[chrom] = frag_grouped


In [ ]:
import pickle
pickle.dump(tile_dict, open("./data/scatac/processed_input/{}_tile_50bp_dict.p".format(ct), "wb"))  # save it into a file named save.p
np.save("./data/scatac/processed_input/{}_tile_50bp_barcode.npy".format(ct),barcodes.astype(str))


# Compute metacells

In [ ]:
import sys
from scipy.sparse import coo_matrix, csr_matrix, find
from sklearn.neighbors import NearestNeighbors

In [ ]:
lsi = pd.read_csv('./data/scatac/archr_filtered_lsi.csv', index_col = 0)
lsi.index = [x.split('#')[1] for x in lsi.index]


In [ ]:
def get_max_overlap(y,x):
    return np.max(x.dot(y))
def generate_cicero_metacell(nbrs, max_overlap, sampled_id = [0]):
    order = np.arange(nbrs.shape[0])
    np.random.seed(10)
    np.random.shuffle(order)

    selected_idx = [False]*nbrs.shape[0]
    for i in sampled_id:
        selected_idx[i] = True
    
    for idx in order:
        selected_cells = nbrs[selected_idx]
        candidate = nbrs[idx]
        overlap = get_max_overlap(candidate,selected_cells)
        if overlap < max_overlap:
            selected_idx[idx] = True
            
    return selected_idx
    

In [ ]:
from sklearn.neighbors import NearestNeighbors
n_neighbors = 100
max_overlap = 33

nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean').fit(lsi.values)
nbrs = nbrs.kneighbors_graph(lsi.values).toarray()
selected_idx = generate_cicero_metacell(nbrs, max_overlap = max_overlap)


In [ ]:
metacell_assignment = nbrs[selected_idx,:]
